Alongside the TCM you have implemented, you have also implemented
 assumptions about the physics of the world. Namely, you have assumed that
 the world can be represented as a set of draws from N independent Gaussians, and
 that the means of these Gaussians evolve over time linearly. Now we change this. We
 assume that the world contains context changes, and represent this fact by
 sampling the rate of drift over time (delta) in the feature means itself from a
 mixture of two Gaussians, one with a small mean to denote small changes,
 and one with a large mean to denote large changes. 


Implement this new physics of the world in the model, and design an
 encoding schedule that lets your model retrieve effectively (>7 success).
 The trivial solution is to shove all the encodings towards the back end
 of the encoding period, but doing so will increase your encoding load (proxy for study effort)

 An optimal solution would find the scheduling pattern that minimizes
% encoding load while maintaining average retrieval success across multiple
 runs at 7. See if you can find it, assuming your retrieving agent knows
 the parameters of the model that will generate his world contexts.


In [1]:
# Importing Required libraries
import numpy as np
from drawFromADist import drawFromADist
import matplotlib.pyplot as plt

In [2]:
def get_gaussian_mixture(mean0=0, var0=0.2, mean1=1, var1=0.2, num_of_samples=20000, plot=False):
    y = np.random.normal(mean0, var0, num_of_samples)
    y = np.concatenate((y, np.random.normal(mean1, var1, num_of_samples)))
    if plot:
        plt.hist(y, density=True, bins=50)
        plt.show()
    return y

In [3]:
def model(item_occurance, random_world_m=True, beta_param=0.99):
    N_WORLD_FEATURES = 5
    N_ITEMS = 10
    ENCODING_TIME = 500
    TEST_TIME = 25
    
    schedule_list = np.sort(np.ceil(item_occurance)).reshape(-1, 1)
    item_list = np.array(range(N_ITEMS)).reshape(-1,1)
    schedule = np.concatenate((schedule_list, item_list), axis=1)  # https://numpy.org/doc/stable/reference/generated/numpy.concatenate.html
    schedule_load = ENCODING_TIME/np.median(np.diff(schedule[:,0]))
    encoding = np.zeros((N_ITEMS,N_WORLD_FEATURES+1))
    gaussian_mixture = get_gaussian_mixture(plot=False)
    
    if random_world_m:
        # Generate random world 
        world_m = np.random.randint(low=1, high=5, size=N_WORLD_FEATURES)
    else:
        world_m = np.array([1, 2, 1, 2, 3]) 
    world_var = 1
    #delta = 0.05             
    #beta_param = 0.9  #context drift parameter, ct=beta*ct + c_t-1      # beta_param is in function call
    m = 0
    # simulating encoding
    for time in range(1,ENCODING_TIME + 1):
        delta=np.random.choice(gaussian_mixture) # CT_In
        world_m = beta_param*world_m + delta
        world = np.random.normal(world_m, world_var)

        if(m<N_ITEMS):
            if(time==schedule[m,0]):
                encoding[m] =  np.append(world, m)
                m += 1

    out=np.zeros(TEST_TIME)
    while time < ENCODING_TIME + TEST_TIME:
        delta=np.random.choice(gaussian_mixture)
        world_m = beta_param*world_m + delta

        world = np.random.normal(world_m, world_var)
        
        soa=np.zeros(N_ITEMS)
        current_world=np.zeros(N_WORLD_FEATURES+1)
        for m in range(N_ITEMS):
            current_world=  np.append(world, m)
            
            soa[m]=np.dot(encoding[m],current_world.T)     
        #NORMALIZE
        soa=soa/np.sqrt(np.sum(soa**2)) 
        # REF: https://stackoverflow.com/questions/21030391/how-to-normalize-a-numpy-array-to-a-unit-vector
    
        out[time - ENCODING_TIME] = np.nonzero(drawFromADist(soa))[0][0] # Get the non zero element 
        # REF: https://www.geeksforgeeks.org/numpy-nonzero-in-python/

        time = time + 1
    success=len(np.unique(out))
    return success, schedule_load

We will first try the naive approach of shoving all schedules at the end of the encoding period

In [4]:
out = []
item_occurance=[491,492,493,494,495,496,497,498,499,500]
schedule_load = 0
num_of_trials=50
for i in range(num_of_trials):
    success, schedule_load = model(item_occurance, random_world_m=True)
    out.append(success)
#plt.plot(out)
print(f"Mean of number of items retrieved over {num_of_trials} trials is :{np.mean(out)}. Schedule Load for this schedule:{schedule_load}")


Mean of number of items retrieved over 50 trials is :9.46. Schedule Load for this schedule:500.0


Now we will see schedule load for random schedule

In [5]:
out = []
item_occurance=np.random.random(10) * 500
schedule_load = 0
num_of_trials=50
for i in range(num_of_trials):
    success, schedule_load = model(item_occurance, random_world_m=True)
    out.append(success)
#plt.plot(out)
print(f"Mean of number of items retrieved over {num_of_trials} trials is :{np.mean(out)}. Schedule Load for this schedule:{schedule_load}")


Mean of number of items retrieved over 50 trials is :9.24. Schedule Load for this schedule:11.904761904761905


- We try to minimize schedule load by analysing formula
- Looking at formula of schedule load, we see that we need to maximize np.median(np.diff(schedule)) i.e. the denominator because numerator is constant
- After applying np.diff on a schedule, we will get an array of size 9, from which we take median.
- We need this median to be maximum. 
- Since timing range from 1 to 500, maximum sum of all elements from diff array can be 500 - 1 = 499.
- Now diff array will look like this: _ , _ , _ , _ , Median, _ , _ , _ , _ 
- Here median is at center which needs to be maximized, thus element at the left of median should be as small as possible, and elements at right of median should be equal to median or almost equal to the median


- Let elements at left of median be 1, thus since sum of all elements can be 499, we subract 1*4 from 499, 
- Thus remaining sum = 499 - 4 = 495

- Now by intuition we can see that elements on side of median should all be equal to each other and median too, or be +1 of median at most.
- Thus, median = 495 - 4 * median   
- Thus median = 99
- Thus diff array after sorting should look like this : 1, 1, 1, 1, 99, 99 ,99 ,99 ,99

- Now we can use this diff array and backtrack to find possible original array. Diff was found by using out[i] = arr[i+1] - arr[i]
- Thus we can start with an initial value for arr[0] then find whole array.
- There can be many possible schedule, we can pick elements of diff array in any order.
- One of the schedule with initial element at 1:
- 1, 2(1+1), 3(2+1), 4(3+1), 5(4+1), 104(5+99), 203(104+99), 302(203+99), 401(302+99), 500(401+99)

In [6]:
out = []
item_occurance = [1,2,3,4,5,104,203,302,401,500]
schedule_load = 0
num_of_trials=50
for i in range(num_of_trials):
    success, schedule_load = model(item_occurance, random_world_m=False, beta_param=0.99)
    out.append(success)
#plt.plot(out)
print(f"Mean of number of items retrieved over {num_of_trials} trial is :{np.mean(out)}. Schedule Load for this schedule:{schedule_load}")


Mean of number of items retrieved over 50 trial is :6.4. Schedule Load for this schedule:5.05050505050505


We can have another schedule as:
1, 100(1+99), 199(100+99), 298(199+99), 397(298+99), 496(397+99), 497(496+1), 498, 499, 500, whose schedule load will be minimum too

In [7]:
out = []
item_occurance = [1, 100, 199, 298, 397, 496, 497, 498, 499, 500]
schedule_load = 0
num_of_trials=50
for i in range(num_of_trials):
    success, schedule_load = model(item_occurance, random_world_m=True, beta_param=0.99)
    out.append(success)
#plt.plot(out)
print(f"Mean of number of items retrieved over {num_of_trials} trial is :{np.mean(out)}. Schedule Load for this schedule:{schedule_load}")


Mean of number of items retrieved over 50 trial is :8.68. Schedule Load for this schedule:5.05050505050505


We see that even when schedule load is minimized, occurance of order matters, when more items are occuring towards the end instead of beginnning for maximizing schedule load, we get higher retrieval, this is because our beta param = 0.99, it means world is accomodating change but it also give importance to past state of world.